In [1]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [2]:
import numpy as np
import time
import pickle
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

from package.plots import diagnostic_plot, plot, lens_plot
from package.helpers.lens import Lens
from package.helpers import pyhalo

import datetime
import time

import numpy as np
from lenstronomy.SimulationAPI.ObservationConfig.Roman import Roman
from lenstronomy.Util import data_util
from pandeia.engine.calc_utils import build_default_calc, build_default_source
from pandeia.engine.perform_calculation import perform_calculation
from tqdm import tqdm

from package.helpers.roman_params import RomanParameters
from package.pandeia import pandeia_input, pandeia_output

plt.style.use(f'{repo_path}/package/mplstyle/science.mplstyle')

In [3]:
csv = os.path.join(repo_path, 'data', 'roman_spacecraft_and_instrument_parameters.csv')
array_dir = os.path.join(repo_path, 'output', 'arrays', 'skypy_test')
data_dir = os.path.join('/data','bwedig', 'roman-population', 'data')
figure_dir = os.path.join(repo_path, 'figures')
csv_path = os.path.join(data_dir, 'dictparaggln_Area00000010.csv')
df = pd.read_csv(csv_path)

In [4]:
index = 3
row = df.iloc[index]

lens = Lens(z_lens=row['redslens'], 
            z_source=row['redssour'], 
            sigma_v=row['velodisp'], 
            lens_x=row['xposlens'], 
            lens_y=row['yposlens'], 
            source_x=row['xpossour'], 
            source_y=row['ypossour'], 
            mag_lens=row['magtlensF106'], 
            mag_source=row['magtsourF106'])

# add CDM subhalos
lens.add_subhalos(*pyhalo.generate_CDM_halos(lens.z_lens, lens.z_source))

array = lens.get_array(num_pix=90, side=10.)

In [5]:
band = 'f106'

calc = build_default_calc('roman', 'wfi', 'imaging')

# set scene size settings
calc['configuration']['dynamic_scene'] = True
# calc['configuration']['max_scene_size'] = 5

# set filter
calc['configuration']['instrument']['filter'] = band.lower()  # e.g. 'f106'

# turn on noise sources
calc['calculation'] = pandeia_input.get_calculation_dict(init=True)

Convert magnitudes

In [6]:
source_flux_cps = lens.source_model_class.total_flux(lens.kwargs_source_amp)
lens_flux_cps = lens.lens_light_model_class.total_flux(lens.kwargs_lens_light_amp)

total_flux_cps = np.sum(source_flux_cps, lens_flux_cps)

TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
# do Pandeia calculation        
results, execution_time = pandeia_input.get_pandeia_results(calc)

pandeia_output = pandeia_output.PandeiaOutput(results)